### KNN Best Model 
- RandomizedSearchCV params :{'p': 2, 'n_neighbors': 26} / preict_prba_val :0.7621679858775501

- GridSearchCV score : params : {'n_neighbors': 33, 'p': 2} / predict_proba_val :0.76828492532389


  =>RandomizedSearchCV 에서 찾은 p:2 와 n_neighbors 26이 들어 갈 수있는 범위를 만들어 준다 

In [1]:
import pandas as pd 
import numpy as np
data =pd.read_csv("preprocessing\cs_data.csv",
                 header=0,
                 index_col=0)

In [2]:
# NA를 전부 없앤 값 = datap
data.dropna(inplace=True)
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
data.shape


(120269, 11)

In [6]:
# drop na
data.isnull().sum()


SeriousDlqin2yrs                        0
RevolvingUtilizationOfUnsecuredLines    0
age                                     0
NumberOfTime30-59DaysPastDueNotWorse    0
DebtRatio                               0
MonthlyIncome                           0
NumberOfOpenCreditLinesAndLoans         0
NumberOfTimes90DaysLate                 0
NumberRealEstateLoansOrLines            0
NumberOfTime60-89DaysPastDueNotWorse    0
NumberOfDependents                      0
dtype: int64

In [7]:
######################### data 로 dataset 구분 
X= data.drop(columns='SeriousDlqin2yrs')
y= data['SeriousDlqin2yrs']
X.shape , y.shape

((120269, 10), (120269,))

In [8]:
# 1.dataset 구분
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size =0.2, stratify=y ,random_state=0) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=0)

In [9]:
# 2. scaling
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train_scaled1=sc.fit_transform(X_train)
X_val_scaled1=sc.transform(X_val)
X_test_scaled1=sc.transform(X_test)

In [ ]:
######## KNN best model 찾기 ############ 

In [18]:
knn.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

In [23]:
#RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn =KNeighborsClassifier()
param={
    "n_neighbors": range(1,30),
    "p":[1,2]
}
rs=RandomizedSearchCV(knn,param, scoring ='roc_auc', n_iter=10, cv= 5, n_jobs=-1)



In [24]:
rs.fit(X_train_scaled1,y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
                   param_distributions={'n_neighbors': range(1, 30),
                                        'p': [1, 2]},
                   scoring='roc_auc')

In [30]:
rs.best_score_

0.7432778580587763

In [31]:
rs.best_params_

{'p': 2, 'n_neighbors': 26}

In [36]:
best_model=rs.best_estimator_

In [37]:
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score
roc_auc_score(y_val, best_model.predict_proba(X_val_scaled1)[:,1])


0.7621679858775501

In [ ]:
#RandomizedSearchCV score : 0.7432778580587763 / params :{'p': 2, 'n_neighbors': 26} / preict_prba_val :0.7621679858775501

In [1]:
#GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn =KNeighborsClassifier()
param={
    "n_neighbors": range(25,35), # 26이 들어 갈 수 있는 범위 
    "p":[2] 
}
gs=GridSearchCV(knn,param,scoring ='roc_auc', cv= 5, n_jobs=-1)  

In [29]:
gs.fit(X_train_scaled1,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(25, 35), 'p': [2]},
             scoring='roc_auc')

In [33]:
gs.best_score_

0.7476968529356945

In [34]:
gs.best_params_

{'n_neighbors': 33, 'p': 2}

In [39]:
best_model1=gs.best_estimator_
best_model1

KNeighborsClassifier(n_neighbors=33)

In [40]:
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score
roc_auc_score(y_val, best_model1.predict_proba(X_val_scaled1)[:,1])

0.768284925323895

In [ ]:
# GridSearchCV score : 0.7476968529356945 /params : {'n_neighbors': 33, 'p': 2} / predict_proba_val :0.768284925323895